In [1]:
from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import openai
import time
from langchain.memory import ConversationBufferMemory

In [17]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_BASE =  os.getenv('OPENAI_API_BASE') 
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION') 
OPENAI_API_TYPE = os.getenv('OPENAI_API_TYPE') 

openai.api_type = OPENAI_API_TYPE
openai.api_base = OPENAI_API_BASE 
openai.api_version = OPENAI_API_VERSION
openai.api_key = OPENAI_API_KEY 

In [11]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback


def process_text(text):
    # Split the text into chunks using Langchain's CharacterTextSplitter
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=10000,
        chunk_overlap=1000,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    
    # Convert the chunks of text into embeddings to form a knowledge base
    embeddings = OpenAIEmbeddings(engine="text-embedding-ada-002") 
    knowledgeBase = FAISS.from_texts(chunks, embeddings)
    
    return knowledgeBase

text_folder = r'C:\Users\pkwong\OneDrive - Canada Pension Plan Investment Board\Desktop\AI\text'
    
for fn in os.listdir(text_folder):
    pdf_reader = PdfReader(os.path.join(text_folder, fn))
    # Text variable will store the pdf text
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    
    # Create the knowledge base object
    knowledgeBase = process_text(text)
    time.sleep(30)
    


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-07-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 22 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-07-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 18 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to

In [13]:
index = knowledgeBase.as_retriever()

In [18]:
query = "Why do I have to file taxes?"
docs = index.get_relevant_documents(query)

from langchain.chat_models import AzureChatOpenAI
llm = AzureChatOpenAI(temperature=0, verbose=True, deployment_name="gpt-4-32k", openai_api_version="2023-07-01-preview")

chain = load_qa_chain(llm, chain_type='stuff')
with get_openai_callback() as cost:
    response = chain.run(input_documents=docs, question=query)
    print(response)

Filing taxes is required by law for most individuals and businesses. The tax money collected is used by the government to fund public services, infrastructure, social programs, defense, and other government functions. If you earn income above a certain threshold, you are required to file a tax return. The specific amount varies based on factors such as your age, filing status, and type of income. Not filing taxes when required can result in penalties and legal consequences.
